In [1]:
import numpy as np
from scipy import optimize
import pickle
import seaborn as sns
%matplotlib inline

In [2]:
with open('alpha.pkl', 'rb') as f:
    alpha = pickle.load(f)
with open('sigma.pkl', 'rb') as f:
    sigma = pickle.load(f)

In [48]:
n_ins = alpha.shape[0]
n_ins = 500
# alpha_s = alpha[:n_ins]
# sigma_s = sigma[:n_ins, :n_ins]
alpha_s = alpha[-n_ins:]
sigma_s = sigma[-n_ins:, -n_ins:]

gamma, lambd, L, U, dlt, lev = 1, 0.1, -0.2, 0.2, 0, 2
w_old = np.ones(n_ins) / n_ins

In [66]:
%%time
def objective(w):
    return 0.5 * gamma * w.T.dot(sigma_s).dot(w) - w.dot(alpha_s) + lambd * np.sum((w - w_old)**2)


cons = ({'type':'eq',   'fun':lambda x: np.sum(x)-1}, # sum(w_i) = 1
        {'type':'ineq', 'fun':lambda x: lev - np.sum(abs(x))}, # sum(|w_i|) <= Lev # XXX
        {'type':'ineq', 'fun':lambda x: U-x}, # x_i <= U
        {'type':'ineq', 'fun':lambda x: x-L}, # x_i >= L
        )

#cons = ()
res = optimize.minimize(objective, w_old, method='slsqp', tol=1e-3, constraints=cons)

CPU times: user 1min 30s, sys: 1.47 s, total: 1min 31s
Wall time: 50.2 s


In [67]:
res.status

9

In [68]:
res.message

'Iteration limit exceeded'

In [69]:
sum(abs(res.x))

2.0139066106752908